In [ ]:
%%bash
set -e
%%bash
set -e

cd /content

# Download micromamba
curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba

# Create clean Python 3.11 environment
./bin/micromamba create -y -n cb311 -c conda-forge python=3.11 pip

echo "✅ Environment ready: cb311"

In [ ]:
%%bash
set -euo pipefail

cd /content
MICROMAMBA="/content/bin/micromamba"

echo "[$(date)] Installing PyTorch 2.5.1 (CUDA 12.1)..."
"$MICROMAMBA" run -n cb311 pip install -U pip setuptools wheel
"$MICROMAMBA" run -n cb311 pip install \
  torch==2.5.1+cu121 torchaudio==2.5.1+cu121 torchvision==0.20.1+cu121 \
  --index-url https://download.pytorch.org/whl/cu121

echo "[$(date)] Installing chatterbox-tts (latest from GitHub)..."
"$MICROMAMBA" run -n cb311 pip uninstall -y chatterbox-tts || true
"$MICROMAMBA" run -n cb311 pip install --no-cache-dir --upgrade \
  "git+https://github.com/resemble-ai/chatterbox.git"

echo "[$(date)] Installing extra dependencies..."
"$MICROMAMBA" run -n cb311 pip install onnx==1.16.0 scipy nltk

echo "✅ All packages installed!"

In [ ]:
%%bash
set -e

cd /content

# Remove old version
rm -rf Chatterbox-TTS-Server-main

# Clone YOUR forked repo with multilingual support
git clone https://github.com/Faheem-saif/Chatterbox-TTS-Server-main.git

cd Chatterbox-TTS-Server-main

echo "✅ Your multilingual server cloned!"
ls -la

In [ ]:
import os
import subprocess
import time
import socket
import requests
from pathlib import Path

PORT = 8004
REPO_DIR = "/content/Chatterbox-TTS-Server-main"
LOG_STDOUT = "/content/server.log"

os.chdir(REPO_DIR)

# Install server dependencies
print("Installing server requirements...")
subprocess.run([
    "/content/bin/micromamba", "run", "-n", "cb311",
    "pip", "install", "-r", "requirements-nvidia.txt"
], check=True)

# Clear old log
Path(LOG_STDOUT).unlink(missing_ok=True)

print(f"\nStarting multilingual TTS server on port {PORT}...")
print("Log file:", LOG_STDOUT)

env = os.environ.copy()
env["PYTHONUNBUFFERED"] = "1"
env["HF_HOME"] = "/content/hf_cache"
Path(env["HF_HOME"]).mkdir(exist_ok=True)

proc = subprocess.Popen(
    ["/content/bin/micromamba", "run", "-n", "cb311", "python", "-u", "server.py"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
    env=env
)

# Live log streaming + auto-open UI
with open(LOG_STDOUT, "w") as f:
    shown = False
    while True:
        line = proc.stdout.readline()
        if line:
            print(line, end="")
            f.write(line)
            f.flush()

            if not shown and "Uvicorn running on" in line:
                shown = True
                print("\n" + "="*60)
                print("🎉 SERVER IS READY! Opening Web UI...")
                print("="*60)
                from google.colab import output
                output.serve_kernel_port_as_window(PORT, path="/")

        if proc.poll() is not None:
            print("\nServer stopped.")
            break

In [ ]:
%%bash
PORT=8004

echo "Killing processes on port $PORT..."
sudo lsof -t -i:$PORT | xargs -r sudo kill -9 || true

echo "Done. Server stopped."